# Task Arrays for Data Parallelism
Le calcul haute-performance consiste non seulement au calcul parallèle par tâche (***parallélisme des tâches***),
mais aussi au calcul de données en parallèle dans plusieurs tâches et/ou processus en simultané (***parallélisme de données***).
Ce chapitre vous donnera les outils nécessaires pour gérer un grand nombre de tâches
lorsque le projet de recherche requiert plusieurs centaines de résultats.

## GNU Parallel
La commande `parallel` de
[GNU Parallel](https://docs.alliancecan.ca/wiki/GNU_Parallel/fr)
permet d'utiliser pleinement les ressources locales d'un noeud de
calcul, et ce, en gérant l'exécution d'une **longue liste de tâches
de *petite* taille**.
C'est un peu comme l'ordonnanceur Slurm, mais à plus petite échelle et
en gérant des processus au lieu de scripts de tâche.

![Fonctionnement de GNU Parallel](images/gnu-parallel.svg)

* [Documentation officielle](https://www.gnu.org/software/parallel/parallel.html)
* [Tutoriel](https://www.gnu.org/software/parallel/parallel_tutorial.html)

### Why Not Slurm?
OK, mais pourquoi ne pas tout simplement soumettre
**des centaines de tâches à Slurm**?
* À tout moment, Slurm **limite chaque usager à 1000 tâches**
  au total dans `squeue` (*pending* + *running*)
* Certains calculs sont tellement **courts (< 5 minutes)** que le
  démarrage et la fin de la tâche compteraient pour un pourcentage
  significatif du temps réel utilisé, ce qui diminue leur efficacité

Les avantages de GNU Parallel à considérer :
* Nous **évite d'utiliser une boucle** soumettant des centaines de
  scripts similaires, ce qui, dans bien des cas, facilite
  l'exécution de centaines de cas de calcul semblables
* Le nombre de **processeurs disponibles limite** automatiquement le
  nombre de cas de calcul exécutés en simultané
  * Dans le cas de calculs parallèles, c'est possible de spécifier
    le nombre de cas en simultané
* GNU Parallel peut
  [reprendre la séquence des cas de calcul](https://docs.alliancecan.ca/wiki/GNU_Parallel/fr#Suivi_des_commandes_ex.C3.A9cut.C3.A9es_ou_des_commandes_ayant_.C3.A9chou.C3.A9.3B_fonctionnalit.C3.A9s_de_red.C3.A9marrage)
  en situation de fin hâtive de la tâche Slurm

### GNU Parallel Command Syntax
The main basic elements of a `parallel` command are:
```Bash
parallel options command_template ::: list of values
```

See the manual page:
```Bash
man parallel  # Press Q to quit
```

### Use Cases
#### One Sequence of Parameter Values
The default placeholder for the changing parameter is `{}`:
```Bash
parallel echo file{}.txt ::: 1 2 3 4
# parallel --citation  # Commit to citing developers
```

We can rewrite the above command by using the Bash expansion `{a..b}`:
```Bash
parallel echo file{}.txt ::: {1..4}
parallel echo file{}.txt ::: {01..10}
```

#### Multiple Combinations of Parameter Values
**a)** When you have **multiple sequences of parameters to combine**,
you can use numbered placeholders, like `{1}` and `{2}`:
```Bash
parallel echo file{1}{2}.txt ::: {01..10} ::: a b
```

**b)** In the case where **all combinations
of parameters are in a text file**:
```Bash
cat scripts/param.txt
```

The `parallel` command will have `-C ' '` to specify
the separator between parameters in `scripts/param.txt`,
and the `::::` argument to provide this filename:
```Bash
# parallel -C ' ' echo '$(({1}*{2}))' :::: scripts/param.txt
cat scripts/prll-exec-param.sh
sbatch scripts/prll-exec-param.sh
```

**c)** If you prefer to validate a **list of commands in a
text file** prior to their execution on the compute node:
```Bash
cat scripts/cmd.txt
```

The job script will have a much simplified `parallel` command:
```Bash
# parallel < scripts/cmd.txt
cat scripts/prll-exec-cmd.sh
sbatch scripts/prll-exec-cmd.sh
```

#### Limiting the Number of Simultaneous Processes
The flag `--jobs` allows us to limit the number of
simultaneous running processes. For example, we can
have 8 tasks done with 2 simultaneous processes:
```Bash
parallel --jobs 2 'echo {} && sleep 3' ::: {1..8}
```

### **Exercise** - Aligning DNA Sequences
In the directory `data/`, you should already have multiple
Fasta files (`*.fa`) that were created in the previous
chapter with the job script `scripts/blastn-gen-seq.sh`.
**If this is not the case**, run the following:
```Bash
sbatch scripts/blastn-gen-seq.sh
```

You should find:
* for each fictive species A, B, C and D,
  a file of "known" DNA sequences
  * These files are converted into a Blast database
* for 16 "unknown" species K through Z, DNA sequences
  to align on "known" sequences of species A through D

We then want to compute the alignement of all
combinations `{A,B,C,D}` x `{K,L,M,N,O,P,Q,R,S,T,U,V,W,X,Y,Z}`,
which makes 64 combinations to test.
The following job script uses GNU Parallel in order
to compute all combinations with 4 CPU cores:
```Bash
cat scripts/blastn-parallel.sh
sbatch scripts/blastn-parallel.sh
```

To monitor the status of the compute job:
```Bash
squeue -u $USER
ls data/res_prll/
```
At the end of the job, check used resources:
```
seff <job_id>
```

### Other Tools
* GLOST
  [pour des calculs séquentiels seulement](https://docs.alliancecan.ca/wiki/GLOST/fr)

* META-Farm
  [pour le meilleur de GNU Parallel et GLOST](https://docs.alliancecan.ca/wiki/META-Farm/fr)

Alors que les précédents outils s'utilisent bien avec un lot de
calculs séquentiels ou parallèles de petite taille (16 processeurs
ou moins), **ils ne sont pas** vraiment **appropriés pour**
un lot de **longs calculs parallèles de plus grande taille**
(plus de 16 processeurs par calcul) :
* on veut éviter les longues tâches qui dépassent trois (3) jours et
* on veut réduire le risque de subir une défaillance matérielle.

C'est pourquoi, dans certains cas, il vaut
mieux utiliser les vecteurs de tâches.

## Job Arrays
Dans le cas où un même programme doit être exécuté avec différentes
combinaisons de paramètres, il y a moyen de soumettre un seul
[vecteur de tâches](https://docs.alliancecan.ca/wiki/Job_arrays/fr)
et de coder le script de tâche de telle sorte que les paramètres
seront déterminés **en fonction d'un indice unique** du vecteur de tâches.

![How Job Arrays Work](images/job-arrays.svg)

**Pour soumettre un vecteur de tâches** à l'ordonnanceur Slurm, que ce
soit à la ligne de commande `sbatch` ou dans l'entête `#SBATCH`
du script de tâche, **on doit ajouter l'option** `--array=<indices>`.
Voir [ici quelques exemples](https://docs.alliancecan.ca/wiki/Job_arrays/fr).

L'identifiant d'une tâche Slurm dans un vecteur de tâches contient :
* L'identifiant du vecteur de tâches
* Caractère de soulignement (`_`)
* L'indice unique associé à la tâche

**Par exemple :** `25249551_15`

Dans le script de tâche, la **variable d'environnement**
`$SLURM_ARRAY_TASK_ID` peut être utilisée pour retrouver la valeur
actuelle de l'indice unique associé à la tâche en cours d'exécution.
Il s'agit d'un **entier parmi** les `<indices>`.

The variable `$SLURM_ARRAY_TASK_ID` can be used in many ways.
The below examples use `$N`, but `$SLURM_ARRAY_TASK_ID`
works the same in a job script:
```Bash
export N=71  # Only required for this example

echo file.$N
echo directory-$N
```

![From One Integer to 2D Coordinates](images/n2r_c.svg)
```Bash
PARAM_R=$((N / 12))  # Integer division
PARAM_C=$((N % 12))  # Modulo (division remainder)
echo $PARAM_R $PARAM_C

head -n $((PARAM_R + 1)) scripts/param.txt | tail -1
```

### **Exercise** - Job Arrays
Submit the job array:
```Bash
cat scripts/blastn-array.sh
sbatch scripts/blastn-array.sh

squeue -u $USER
```

Once all four jobs are done, inspect the results:
```Bash
ls slurm-*_*.out
ls -l data/res_array/
```

* **Modify the job script** to process all 16 unknowns K through Z,
  with a limit of **four** simultaneous processes per job
* Submit this modified job array
* Use the `seff` command to investigate one of the 16 jobs

## Key Points
* **GNU Parallel**
  [to run multiple combinations of parameters](https://docs.alliancecan.ca/wiki/GNU_Parallel)

```Bash
parallel 'command_template({1})' ::: values1
parallel 'command_template({1}, {2})' ::: values1 ::: values2
parallel -C <sep> 'command_template({1}, {2})' :::: param_pairs.txt
parallel --jobs 'N_processes_per_node' < command_list.txt
```

* **Job Arrays**
  [to submit many long or large similar jobs](https://docs.alliancecan.ca/wiki/Job_arrays)

```Bash
# $SLURM_ARRAY_TASK_ID will have only one value of ...
sbatch --array=0-7      # [0, 7]
sbatch --array=1,3,5,7  # {1,3,5,7}
sbatch --array=1-7:3    # {1,4,7}
sbatch --array=0-99%10  # [0, 99], but maximum 10 simultaneous jobs
```